# Final Individual Recommender System

# Importing the Datasets

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Make Pandas Display all columns
pd.set_option('display.max_columns', None)

In [3]:
# Load the ratings data
ratings_df_slim = pd.read_csv('../dataset/XWines_Slim_150K_ratings.csv')
ratings_df_full = pd.read_csv('../dataset/XWines_Full_21M_ratings.csv')

# Load the wines data
wines_df_slim = pd.read_csv('../dataset/XWines_Slim_1K_wines.csv')
wines_df_full = pd.read_csv('../dataset/XWines_Full_100K_wines.csv')


/var/folders/9n/hfw66rw97sl45x98mmd_vs4r0000gn/T/ipykernel_56438/1459283311.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df_slim = pd.read_csv('../dataset/XWines_Slim_150K_ratings.csv')
/var/folders/9n/hfw66rw97sl45x98mmd_vs4r0000gn/T/ipykernel_56438/1459283311.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_df_full = pd.read_csv('../dataset/XWines_Full_21M_ratings.csv')


# Data Pre-Processing

In [4]:
# Dropping some of the Columns in the wines_df
# Some are irrelevant for analysis (like the website) and some will just be repeated info (like the Code and Country)
wine_columns_to_drop_repeated = ['Website', 'Code', 'RegionID', 'WineryID', 'WineName', 'RegionName', 'WineryName']
wines_df_slim.drop(columns=wine_columns_to_drop_repeated, inplace=True)
wines_df_full.drop(columns=wine_columns_to_drop_repeated, inplace=True)

In [5]:
# Dropping the Features we will not use for ML due to the extremely high dimensionality
wine_columns_to_drop_high_dimensionality = ['Grapes'] # Too many unique values
wines_df_slim.drop(columns=wine_columns_to_drop_high_dimensionality, inplace=True)
wines_df_full.drop(columns=wine_columns_to_drop_high_dimensionality, inplace=True)

In [6]:
# Convert string to list
import ast

wines_df_slim['Harmonize'] = wines_df_slim['Harmonize'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
wines_df_slim['Vintages'] = wines_df_slim['Vintages'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
wines_df_slim['Vintages'] = wines_df_slim['Vintages'].apply(lambda x: [int(v) for v in ast.literal_eval(x)] if isinstance(x, str) else x)

wines_df_full['Harmonize'] = wines_df_full['Harmonize'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
wines_df_full['Vintages'] = wines_df_full['Vintages'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
wines_df_full['Vintages'] = wines_df_full['Vintages'].apply(lambda x: [int(v) for v in ast.literal_eval(x)] if isinstance(x, str) else x)

In [7]:
# Processing Vintages
def process_vintages(vintages):
    # Remove "N.V." from the list
    filtered_vintages = [vintage for vintage in vintages if vintage != "N.V."]

    # Calculate the number of vintages
    num_vintages = len(filtered_vintages)

    # Find the oldest and most recent vintages
    oldest_vintage = min(filtered_vintages) if filtered_vintages else 0
    most_recent_vintage = max(filtered_vintages) if filtered_vintages else 0
    vintage_range = most_recent_vintage - oldest_vintage

    return num_vintages, oldest_vintage, most_recent_vintage, vintage_range

# Apply the function to each row for both dataset versions
wines_df_slim[['num_vintages', 'oldest_vintage', 'most_recent_vintage', 'vintage_range']] = wines_df_slim['Vintages'].apply(process_vintages).tolist()
wines_df_full[['num_vintages', 'oldest_vintage', 'most_recent_vintage', 'vintage_range']] = wines_df_full['Vintages'].apply(process_vintages).tolist()

In [8]:
# Drop the original Vintages column for both datasets, since we susbtituted with the new columns 
wines_df_slim.drop(columns=['Vintages'], inplace=True)
wines_df_full.drop(columns=['Vintages'], inplace=True)

In [9]:
#some renaming if we want to use the lenskit later:
ratings_df_slim = ratings_df_slim.rename(columns={'UserID': 'user', 'WineID': 'item', 'Rating': 'rating', 'Date': 'timestamp'})
ratings_df_full = ratings_df_full.rename(columns={'UserID': 'user', 'WineID': 'item', 'Rating': 'rating', 'Date': 'timestamp'})


wines_df_slim = wines_df_slim.rename(columns={'WineID': 'item'})
wines_df_full = wines_df_full.rename(columns={'WineID': 'item'})

In [10]:
# Explode 'Harmonize' column into individual rows
wines_exploded_slim = wines_df_slim.explode('Harmonize')
wines_exploded_full = wines_df_full.explode('Harmonize')

# One-hot encode the 'Harmonize' column with the desired prefix
harmonize_one_hot_slim = pd.get_dummies(wines_exploded_slim['Harmonize'], prefix='harmonize_with')
harmonize_one_hot_full = pd.get_dummies(wines_exploded_full['Harmonize'], prefix='harmonize_with')

# Group by 'item' and sum to combine back into single rows per item
harmonize_one_hot_grouped_slim = harmonize_one_hot_slim.groupby(wines_exploded_slim['item']).sum()
harmonize_one_hot_grouped_full = harmonize_one_hot_full.groupby(wines_exploded_full['item']).sum()

# Merge the one-hot encodings with the original DataFrame
wines_df_slim = wines_df_slim.merge(harmonize_one_hot_grouped_slim, left_on='item', right_index=True, how='left')
wines_df_full = wines_df_full.merge(harmonize_one_hot_grouped_full, left_on='item', right_index=True, how='left')

In [11]:
# Drop the Harmonize Column on both versions for the wines
wines_df_slim.drop(columns=['Harmonize'], inplace=True)
wines_df_full.drop(columns=['Harmonize'], inplace=True)

In [12]:
# Re-apply One-Hot Encoding with dtype=int for wines_df_slim
wines_df_slim = pd.get_dummies(
    wines_df_slim,
    columns=['Type', 'Elaborate', 'Body', 'Acidity', 'Country'],
    dtype=int
)

# Re-apply One-Hot Encoding with dtype=int for wines_df_full
wines_df_full = pd.get_dummies(
    wines_df_full,
    columns=['Type', 'Elaborate', 'Body', 'Acidity', 'Country'],
    dtype=int
)

In [ ]:
# Performing One Hot Encodding for the Categorical Features
wines_df_slim = pd.get_dummies(wines_df_slim, columns=['Type', 'Elaborate', 'Body', 'Acidity', 'Country'])
wines_df_full = pd.get_dummies(wines_df_full, columns=['Type', 'Elaborate', 'Body', 'Acidity', 'Country'])

In [ ]:
wines_df_slim.head()

In [ ]:
# Scale the Numerical Features
numerical_features_to_scale = ['ABV', 'num_vintages', 'oldest_vintage', 'most_recent_vintage', 'vintage_range']

# Scale the numerical features for both datasets
from sklearn.preprocessing import StandardScaler

scaler_for_slim = StandardScaler()
wines_df_slim[numerical_features_to_scale] = scaler_for_slim.fit_transform(wines_df_slim[numerical_features_to_scale])

scaler_for_full = StandardScaler()
wines_df_full[numerical_features_to_scale] = scaler_for_full.fit_transform(wines_df_full[numerical_features_to_scale])

In [ ]:
# How the Wines Dataset looks for ML
wines_df_slim.head()

## Create the Mask for the Ratings
* 1 - 3.5 - 0
* 4 - 4.5 - 1
* 4.5 - 5 - 2

In [ ]:
# Go through the Ratings on both datasets and subsitute the values based on conditions. 
# <=3.5 becomes 0
# 4 becomes 1
# >= 4.5 becomes 2
def label_rating(rating):
    if rating <= 3.5: #~41%
        return 0
    elif rating == 4.0: #~36%
        return 1
    elif 4.5 <= rating <= 5.0: #~22%
        return 2

# Apply the function to create the new column 'rating_label'
ratings_df_slim['rating_label'] = ratings_df_slim['rating'].apply(label_rating)
ratings_df_full['rating_label'] = ratings_df_full['rating'].apply(label_rating)


In [ ]:
ratings_df_slim.head(5)

In [ ]:
wines_df_slim.head()

# Content Based Classifier - Decision Tree

In [ ]:
# Continue from where you left off

# Merge the ratings with the wine features
data = ratings_df_slim.merge(wines_df_slim, on='item')

# List of wine feature columns (exclude 'item' and any other non-feature columns if necessary)
wine_feature_columns = wines_df_slim.columns.tolist()
wine_feature_columns.remove('item')

# Initialize a list to store evaluation metrics
metrics_list = []

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
from tqdm import tqdm

# Get the list of unique users
users = data['user'].unique()

# Iterate over each user
for user_id in tqdm(users, desc='Processing Users'):
    # Get the data for the current user
    user_data = data[data['user'] == user_id]
    
    # Skip users with insufficient data
    if len(user_data) < 5:
        continue  # Skip users with less than 5 ratings
    
    # Prepare features (X) and target variable (y)
    # X should contain only the wine features
    X = user_data[wine_feature_columns]
    y = user_data['rating_label']
    
    # Ensure that X contains only wine features
    # Optionally, print X.columns to verify
    # print("Features used for training:", X.columns.tolist())
    
    # Split into training and testing sets, stratified on 'rating_label'
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
    except ValueError:
        # If stratification is not possible due to class imbalance, skip this user
        continue
    
    # Ensure all features are numerical
    X_train = X_train.apply(pd.to_numeric, errors='coerce').fillna(0)
    X_test = X_test.apply(pd.to_numeric, errors='coerce').fillna(0)
    
    # Train the Decision Tree Classifier with max_depth=3
    clf = DecisionTreeClassifier(max_depth=3, random_state=42)
    clf.fit(X_train, y_train)
    
    # Get the items not rated by the user
    user_rated_items = user_data['item'].unique()
    all_items = wines_df_slim['item'].unique()
    items_to_predict = np.setdiff1d(all_items, user_rated_items)
    
    # Get features for the items to predict
    items_to_predict_df = wines_df_slim[wines_df_slim['item'].isin(items_to_predict)]
    
    # Prepare features for prediction
    X_predict = items_to_predict_df[wine_feature_columns]
    
    # Ensure that the feature columns match those used in training
    missing_cols = set(X_train.columns) - set(X_predict.columns)
    for col in missing_cols:
        X_predict[col] = 0  # Add missing columns with default value 0
    X_predict = X_predict[X_train.columns]  # Reorder columns
    
    # Ensure all features are numerical
    X_predict = X_predict.apply(pd.to_numeric, errors='coerce').fillna(0)
    
    # Predict the 'rating_label' for these items
    y_pred = clf.predict(X_predict)
    
    # Create a DataFrame with the predictions
    predictions_df = pd.DataFrame({
        'item': items_to_predict_df['item'],
        'predicted_rating_label': y_pred
    })
    
    # Map the predicted labels to scores for ranking (higher labels indicate higher preference)
    predictions_df['predicted_score'] = predictions_df['predicted_rating_label']
    
    # Get the top N recommendations
    top_n = 10
    recommendations = predictions_df.sort_values(by='predicted_score', ascending=False).head(top_n)
    
    # Prepare ground truth for evaluation
    # Get the items in the test set with 'rating_label' >= 1 (considered relevant)
    test_items = user_data.loc[y_test.index, 'item'].values
    relevant_items = user_data.loc[y_test.index][user_data['rating_label'] >= 1]['item'].values
    
    # Recommended items
    recommended_items = recommendations['item'].values
    
    # Compute evaluation metrics
    # Precision@N
    hits = [1 if item in relevant_items else 0 for item in recommended_items]
    num_hits = sum(hits)
    precision = num_hits / top_n if top_n > 0 else 0
    
    # Recall@N
    num_relevant_items = len(relevant_items)
    recall = num_hits / num_relevant_items if num_relevant_items > 0 else 0
    
    # NDCG@N
    relevance_scores = [1 if item in relevant_items else 0 for item in recommended_items]
    if sum(relevance_scores) > 0:
        ndcg = ndcg_score([relevance_scores], [relevance_scores], k=top_n)
    else:
        ndcg = 0.0
    
    # Store the metrics
    metrics_list.append({
        'user': user_id,
        'Precision@{}'.format(top_n): precision,
        'Recall@{}'.format(top_n): recall,
        'NDCG@{}'.format(top_n): ndcg
    })

# Create a DataFrame with the evaluation metrics
metrics_df = pd.DataFrame(metrics_list)

# Compute the average metrics
average_metrics = metrics_df.mean(numeric_only=True)

# Display the average metrics
print("\nAverage Metrics Across All Users:")
for metric_name, value in average_metrics.items():
    print(f"{metric_name}: {value:.4f}")

# Optionally, visualize the distribution of metrics
metrics_melted = metrics_df.melt(
    id_vars='user',
    value_vars=['Precision@{}'.format(top_n), 'Recall@{}'.format(top_n), 'NDCG@{}'.format(top_n)],
    var_name='Metric',
    value_name='Score'
)

plt.figure(figsize=(12, 6))
sns.boxplot(x='Metric', y='Score', data=metrics_melted)
plt.title('Distribution of Evaluation Metrics Across Users')
plt.xlabel('Metric')
plt.ylabel('Score')
plt.tight_layout()
plt.show()
